<a href="https://colab.research.google.com/github/Hmnshuuu/Twitter-Sentimental-Analysis/blob/main/Twitter_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
!pip install kaggle

In [4]:
#configure path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#api to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

 83% 67.0M/80.9M [00:00<00:00, 226MB/s]
100% 80.9M/80.9M [00:00<00:00, 229MB/s]


In [6]:
#extract compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("the data is extracted")

the data is extracted


In [7]:
import re      #regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer           #reduce word to its root word
from sklearn.feature_extraction.text import TfidfVectorizer         #textual values into numerical values
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
df = pd.read_csv('/content/twitter_data.csv',encoding = 'ISO-8859-1')

In [11]:
df.shape

(1599999, 6)

In [12]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
cols_name = ['target','id','date','flag','user','text']
df = pd.read_csv('/content/twitter_data.csv',names = cols_name, encoding = 'ISO-8859-1')

In [14]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
df.shape

(1600000, 6)

In [16]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [18]:
df.replace({'target':{4:1}},inplace = True)

In [19]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)   #remove all other than a-z or A-Z
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [22]:
df['stemmed_content'] = df['text'].apply(stemming)

In [23]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [24]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [26]:
X = df['stemmed_content'].values
Y = df['target'].values

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,stratify = Y,random_state =2)

In [28]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [30]:
# make the model
model = LogisticRegression(max_iter = 1000)

In [31]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [32]:
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_pred)

In [33]:
print('accuracy on training data is :',training_data_accuracy)

accuracy on training data is : 0.81018984375


In [34]:
X_test_pred = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test,X_test_pred)

In [35]:
print('accuracy on testing data is :',testing_data_accuracy)

accuracy on testing data is : 0.7780375


In [36]:
#saving the trianed model
import pickle

In [37]:
filename = 'trained_sentimental_analysis_model.sav'
pickle.dump(model,open(filename,'wb'))

In [38]:
#using saved model for future predictions
loaded_model = pickle.load(open('/content/trained_sentimental_analysis_model.sav','rb'))

In [39]:
X_new = X_test[200]
print(Y_test[200])

1


In [40]:
prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

[1]
positive tweet


In [41]:
prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

[1]
positive tweet


In [42]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

0
[0]
negative tweet


In [43]:
X_new = X_test[323]
print(Y_test[323])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

1
[1]
positive tweet
